In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [2]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:3] # 새로운 샘플이라고 생각합니다.

Epoch 1/20
363/363 [==============================] - 2s 5ms/step - loss: 1.3464 - val_loss: 0.5414
Epoch 2/20
363/363 [==============================] - 2s 5ms/step - loss: 0.5240 - val_loss: 0.4881
Epoch 3/20
363/363 [==============================] - 3s 8ms/step - loss: 0.5095 - val_loss: 0.4759
Epoch 4/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4554 - val_loss: 0.4495
Epoch 5/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4352 - val_loss: 0.4423
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.4248 - val_loss: 0.4264
Epoch 7/20
363/363 [==============================] - 1s 4ms/step - loss: 0.4211 - val_loss: 0.4255
Epoch 8/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4155 - val_loss: 0.4214
Epoch 9/20
363/363 [==============================] - 2s 4ms/step - loss: 0.4060 - val_loss: 0.4142
Epoch 10/20
363/363 [==============================] - 2s 5ms/step - loss: 0.4013 - val_loss: 0.4108

In [3]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [4]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [6]:
model.compile(loss="mse", optimizer = keras.optimizers.SGD(lr=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

c:\Users\JunHuiLim\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 2.1166 - val_loss: 0.7827
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6880 - val_loss: 0.6050
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5979 - val_loss: 0.5693
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5642 - val_loss: 0.5395
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5417 - val_loss: 0.5219
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.5253 - val_loss: 0.5101
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5123 - val_loss: 0.4998
Epoch 8/20
363/363 [==============================] - 1s 1ms/step - loss: 0.5036 - val_loss: 0.4929
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4952 - val_loss: 0.4866
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4891 - val_loss: 0.4831

In [7]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # 표준 매개변수를 처리합니다.(예를 들면, name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
    
model = WideAndDeepModel()

AttributeError: module 'keras.api._v2.keras' has no attribute 'model'